In [30]:
import RNA
import random

def read_fasta(file_path):
    sequences = {}
    with open(file_path) as f:
        seqname = ''
        seq = ''
        for line in f:
            line = line.strip()
            if line.startswith('>'):
                if seqname:
                    sequences[seqname] = seq
                seqname = line.lstrip('>')
                seq = ''
            else:
                seq += line
        if seqname:
            sequences[seqname] = seq
    return sequences  

def get_mfe_structure(seq):
    """Fold RNA sequence and return MFE structure"""
    fc = RNA.fold_compound(seq)
    structure, mfe = fc.mfe()
    return structure

def filtersequences(sequences, L=30):
    """Filter sequences by length and compute their MFE structures"""
    filtered_sequences = {}
    filtered_structures = {}
    for seqname, seq in sequences.items():
        if len(seq) <= L:
            filtered_sequences[seqname] = seq
            filtered_structures[seqname] = get_mfe_structure(seq)
    return filtered_sequences, filtered_structures


def sample_by_structure(sequences, structures, total_samples=100):
    """
    Group sequences by their MFE structure and sample a total of N sequences
    uniformly at random from all structures.
    
    Parameters:
    sequences: dict of sequence_name -> sequence
    structures: dict of sequence_name -> structure
    total_samples: total number of sequences to sample (default 100)
    
    Returns:
    structure_groups: dict mapping structure -> list of (seqname, seq) tuples
    sampled: dict mapping structure -> sampled sequences
    """
    from collections import Counter
    
    # Group sequences by structure
    structure_groups = {}
    for seqname, seq in sequences.items():
        struct = structures[seqname]
        if struct == len(seq)*'.': continue  # Skip unpaired structures
        if struct not in structure_groups:
            structure_groups[struct] = []
        structure_groups[struct].append((seqname, seq))
    
    # Flatten all sequences and sample total_samples
    all_sequences = []
    for seq_list in structure_groups.values():
        all_sequences.extend(seq_list)
    
    sampled_list = random.sample(all_sequences, min(total_samples, len(all_sequences)))
    
    # Regroup sampled sequences by structure
    sampled = {}
    for seqname, seq in sampled_list:
        struct = structures[seqname]
        if struct not in sampled:
            sampled[struct] = []
        sampled[struct].append((seqname, seq))
    
    return structure_groups, sampled

In [31]:
import random

sequences = read_fasta("/Users/paulagarciagalindo/Library/CloudStorage/OneDrive-UniversityofCambridge/RNA Plasticity/EvoTargetFlipping/sequence.fasta")
sequences, structures = filtersequences(sequences)

# Set seed for reproducibility
random.seed(42)

# Sample 100 sequences from all structures
structure_groups, sampled_seqs = sample_by_structure(sequences, structures, total_samples=100)

# Print summary
print(f"Total sequences: {len(sequences)}")
print(f"Unique structures: {len(structure_groups)}")
print(f"Total sampled sequences: {sum(len(v) for v in sampled_seqs.values())}")
print(f"Structures represented in sample: {len(sampled_seqs)}")
print(f"\nStructures in sample:")
for struct, seqs in sorted(sampled_seqs.items(), key=lambda x: len(x[1]), reverse=True):
    print(f"  {struct}: {len(seqs)} sequences")

Total sequences: 276581
Unique structures: 67526
Total sampled sequences: 100
Structures represented in sample: 100

Structures in sample:
  .(((.(...((....)).).))).......: 1 sequences
  ((((((((((...)))))))))): 1 sequences
  (((((((.......))))))): 1 sequences
  ..((((.....((((.....)))))))): 1 sequences
  .(((((........))))).......: 1 sequences
  ..(((.(((((.......))))).)))..: 1 sequences
  ......(((((....)))))..........: 1 sequences
  ((((...........)))): 1 sequences
  .....(((((.(..........).))))): 1 sequences
  ........((((.....)))).: 1 sequences
  (((........)))...((......)).: 1 sequences
  ........((......))....: 1 sequences
  ..((((...)))).((((......)))): 1 sequences
  (((((....))))).............: 1 sequences
  (((...........)))....: 1 sequences
  ((((((.(((....))).)))))): 1 sequences
  (((((((.....))))))): 1 sequences
  ..((((...))))......: 1 sequences
  ((((((........)))))): 1 sequences
  ................((((....)))).: 1 sequences
  .....((((((((......)))).))))..: 1 sequences
 

In [37]:
# List all sampled sequences
print("All sampled sequences:")
print("-" * 80)
sampled_names = []
lengths = []
for struct, seqs in sampled_seqs.items():
    for seqname, seq in seqs:
        sampled_names.append(seqname)
        lengths.append(len(seq))
        print(f"{seqname}: {seq}")

print(f"\nTotal sampled: {len(sampled_names)}")
print(f"\nSequence names list:\n{sampled_names}")
print(f"\nSequence lengths list:\n{set(lengths)}")

All sampled sequences:
--------------------------------------------------------------------------------
FR189075|DQ549184,DQ686853|Piwi-interacting RNA (piRNA): TGCAGAAACTCTGATGACTGTGCCAAAAAC
FR335129||Putative conserved noncoding region (EvoFold): AATTGTTTTGAACCAAGATAATT
FR009369||Putative conserved noncoding region (EvoFold): ATGGCATATTGTAAATGTCAT
FR327157|DQ625265|Piwi-interacting RNA (piRNA): TAGGCAAAAATCGTATGACTTATGTGCC
FR145667|DQ720854|Piwi-interacting RNA (piRNA): TGTTAGCTTGATTTCTGACTTCTCAA
FR087755|DQ556132|Piwi-interacting RNA (piRNA): TGGTGTGGAAAAGGTTCATTTCTTTACAG
FR237695|DQ562674|Piwi-interacting RNA (piRNA): TTTCCNGTAGTGCCTACTATCTTATTCTCC
FR191843||Putative conserved noncoding region (EvoFold): CAGCTTTTCATCAGTGCTG
FR317982|DQ685181|Piwi-interacting RNA (piRNA): TACTTGTAACCCTTTCCTCCCCGAGTTGC
FR158076||Piwi-interacting RNA (piRNA): TTACCTGGGGGAATCGATCCCA
FR229211|DQ747760|Piwi-interacting RNA (piRNA): TGCAGGCCCAAGCAAATCCTCTGGTGGT
FR423987||Fly small RNA: ATAATTTTGGGTGTAGCCG

In [ ]:

# Write sampled sequences to file with T->U conversion
output_file = "/Users/paulagarciagalindo/Desktop/PhenoDist_Viral/phenotypic_distance/phenodistance/RNA20/sampled_sequences.txt"

with open(output_file, 'w') as f:
    for struct, seqs in sampled_seqs.items():
        for seqname, seq in seqs:
            # Convert T to U
            rna_seq = seq.replace('T', 'U')
            f.write(rna_seq + '\n')

print(f"Sequences written to: {output_file}")
print(f"Total sequences written: {len(sampled_names)}")

Sequences written to: /Users/paulagarciagalindo/Desktop/PhenoDist_Viral/phenotypic_distance/phenodistance/RNA20/sampled_sequences.txt
Total sequences written: 100
